In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from torch.optim import AdamW
import pandas as pd

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
cd /content/drive/MyDrive/projectdatacleaning/

/content/drive/MyDrive/projectdatacleaning


In [27]:
import random
import pandas as pd

# Define the number of examples per mistake type
num_examples = 400

# List of popular domains
popular_domains = [
    "gmail.com", "yahoo.com", "outlook.com", "hotmail.com", "aol.com",
    "icloud.com", "mail.com", "zoho.com", "protonmail.com", "gmx.com"
]

# Function to generate a valid email with a popular domain
def generate_valid_email():
    local_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=10))
    domain = random.choice(popular_domains)
    return f"{local_part}@{domain}"

# Function to generate a correct email based on given parts
def generate_correct_email(local_part):
    domain = random.choice(popular_domains)
    return f"{local_part}@{domain}"

# Function to generate emails with specific mistakes
def generate_emails_with_mistakes():
    emails = []

    # Missing "@" Symbol
    for _ in range(num_examples):
        local_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=10))
        domain_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=5))
        incorrect_email = f"{local_part}{domain_part}"
        correct_email = generate_correct_email(local_part)
        emails.append([incorrect_email, correct_email])

    # Multiple "@" Symbols (random number of "@")
    for _ in range(num_examples):
        local_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=10))
        num_at_symbols = random.randint(2, 5)  # Random number of "@" symbols between 2 and 5
        domain_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=5))
        incorrect_email = f"{local_part}{'@' * num_at_symbols}{domain_part}.com"
        correct_email = generate_correct_email(local_part)
        emails.append([incorrect_email, correct_email])

    # Invalid Characters
    invalid_chars = [' ', ',', '!', '#', '$', '%', '^', '&', '*', '(', ')']
    for _ in range(num_examples):
        local_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=10))
        invalid_char = random.choice(invalid_chars)
        domain_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=5))
        incorrect_email = f"{local_part}{invalid_char}{domain_part}@example.com"
        correct_email = generate_correct_email(local_part)
        emails.append([incorrect_email, correct_email])

    # Incorrect Domain
    for _ in range(num_examples):
        local_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=10))
        domain_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=5))
        incorrect_email = f"{local_part}@{domain_part}.com"
        correct_email = generate_correct_email(local_part)
        emails.append([incorrect_email, correct_email])

    # Invalid Top-Level Domain (TLD)
    for _ in range(num_examples):
        local_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=10))
        domain_part = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=5))
        incorrect_email = f"{local_part}@{domain_part}.c"
        correct_email = generate_correct_email(local_part)
        emails.append([incorrect_email, correct_email])

    return emails

# Generate the dataset
emails_with_mistakes = generate_emails_with_mistakes()

# Create a DataFrame and save to CSV
df = pd.DataFrame(emails_with_mistakes, columns=["incorrect_email", "correct_email"])
df.to_csv("emails_with_mistakes.csv", index=False)

print("Dataset with email mistakes and correct emails generated and saved to 'emails_with_mistakes.csv'.")


Dataset with email mistakes and correct emails generated and saved to 'emails_with_mistakes.csv'.


In [8]:
# Define the path to your CSV file
load_path = "emails_with_mistakes.csv"

# Load the dataset from the CSV file
df = pd.read_csv(load_path)

# Display the first few rows of the dataset to verify it loaded correctly
df

,incorrect_email,correct_email
0,q9mpt11os74groh,q9mpt11os7@zoho.com
1,qlkfauxrho6sqjq,qlkfauxrho@mail.com
2,zdjt4fduadipz1b,zdjt4fduad@icloud.com
3,zq6i26unfk3uyar,zq6i26unfk@gmail.com
4,l1nw2uu0tmpb7p9,l1nw2uu0tm@zoho.com
...,...,...
1995,oqioinu6g8@96x2i.c,oqioinu6g8@gmail.com
1996,s6sxxsq36s@2qm8v.c,s6sxxsq36s@icloud.com
1997,vcarw05brr@xqql6.c,vcarw05brr@mail.com
1998,d5wc8az58o@jxug4.c,d5wc8az58o@zoho.com


In [9]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the emails
df['incorrect_email_tokenized'] = df['incorrect_email'].apply(lambda x: tokenizer(x, max_length=128, padding='max_length', truncation=True, return_tensors='pt'))
df['correct_email_tokenized'] = df['correct_email'].apply(lambda x: tokenizer(x, max_length=128, padding='max_length', truncation=True, return_tensors='pt'))

In [10]:
from torch.utils.data import Dataset, DataLoader

class EmailCorrectionDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        incorrect_email = self.data['incorrect_email_tokenized'].iloc[idx]
        correct_email = self.data['correct_email_tokenized'].iloc[idx]

        return {
            "input_ids": incorrect_email['input_ids'].squeeze(),
            "attention_mask": incorrect_email['attention_mask'].squeeze(),
            "labels": correct_email['input_ids'].squeeze()  # For seq2seq tasks
        }

In [11]:
from sklearn.model_selection import train_test_split

# Split the data: 80% train, 20% remaining for validation and test
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Further split the remaining data into 50% validation and 50% test
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [12]:
# Create Dataset objects
train_dataset1 = EmailCorrectionDataset(train_df)
val_dataset = EmailCorrectionDataset(val_df)
test_dataset = EmailCorrectionDataset(test_df)

In [ ]:
train_dataset[1]

In [13]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments

from torch.nn import CrossEntropyLoss
import torch

# Assuming class 0 is for padding tokens, ignore it
criterion = CrossEntropyLoss(ignore_index=0)  # This ignores the padding index in the labels

model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=tokenizer.vocab_size)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").cuda()
        attention_mask = inputs.get("attention_mask").cuda()

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Compute the loss using CrossEntropyLoss and ignoring the padding tokens
        loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))

        # Apply the attention mask to ignore padding tokens
        loss = loss * attention_mask.view(-1)

        # Take the mean of the non-padding losses
        loss = loss.sum() / attention_mask.sum()

        return (loss, outputs) if return_outputs else loss

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Initialize the trainer with the custom loss function
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset1,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
print(model)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [15]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,8.630100,8.332456
2,6.168200,5.732291
3,3.948700,3.447214
4,2.579100,2.362240
5,1.907300,1.808355
6,1.576400,1.528812
7,1.326600,1.389153
8,1.086100,1.289945
9,0.991700,1.278649
10,0.843200,1.304878


TrainOutput(global_step=1500, training_loss=2.430862607002258, metrics={'train_runtime': 729.9136, 'train_samples_per_second': 32.881, 'train_steps_per_second': 2.055, 'total_flos': 2000377368576000.0, 'train_loss': 2.430862607002258, 'epoch': 15.0})

In [16]:
# Evaluate on the test set
trainer.evaluate(test_dataset)

{'eval_loss': 1.4647645950317383,
 'eval_runtime': 1.8371,
 'eval_samples_per_second': 108.865,
 'eval_steps_per_second': 2.177,
 'epoch': 15.0}

In [21]:
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Ensure the model is on the correct device

predicted_emails = []

model.eval()  # Set the model to evaluation mode

for item in test_dataset:
    # Move input tensors to the same device as the model
    input_ids = item['input_ids'].unsqueeze(0).to(device)
    attention_mask = item['attention_mask'].unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Get predictions using beam search (for better sequence generation)
        predictions = torch.argmax(logits, dim=2)  # Consider using beam search here

        # Decode the predicted token IDs back into a sequence of text
        predicted_email = tokenizer.decode(predictions.squeeze().tolist(), skip_special_tokens=True).strip()

        # Post-process the predicted email to remove unwanted spaces or repeated tokens
        predicted_email = " ".join(predicted_email.split())  # Removes extra spaces
        predicted_emails.append(predicted_email)

# Append the predictions to the test dataframe for comparison
test_df['predicted_email'] = predicted_emails

# Display a few examples of predictions vs actual
for i in range(150,199):
    print(f"Incorrect: {test_df['incorrect_email'].iloc[i]}")
    print(f"Correct: {test_df['correct_email'].iloc[i]}")
    print(f"Predicted: {test_df['predicted_email'].iloc[i]}\n")

Incorrect: 64qoe9uzx3)mg2w5@example.com
Correct: 64qoe9uzx3@protonmail.com
Predicted: 8qoe9uzx3 @ yahoo. com. com com com outlook @ outlook yahoo. com.. com @ yahoomail. com. com @ outlookmail. com.. com com

Incorrect: 9e3vcvjf3q@@dgyc9.com
Correct: 9e3vcvjf3q@outlook.com
Predicted: 9e3vcvjf3q @ gmamail. com com com com @ gma com com com com... com @ gmad com com com com com. com com. gmamail com com com com.. com com com

Incorrect: dmfivpk9xn@lh57c.c
Correct: dmfivpk9xn@zoho.com
Predicted: dmfivpk9xn @ hotmail. com com com com @ @ @ @ com com com com com..l com. @ hot. com com.. com com @.. com com com com com

Incorrect: c2c8nxs7kd@ch42l.c
Correct: c2c8nxs7kd@mail.com
Predicted: ##2c8nxs7kd @ gmmail. com com com com com @ com gm com com com com com com com com com com @ gmmail. com com com com comohmail com com.. com

Incorrect: 8g05zxi1s9csg0v
Correct: 8g05zxi1s9@protonmail.com
Predicted: 8g05zxi1s @ gma yahoo. com @ @ @ @ @ @ com @ @ @ @ yahoo com com @ @ @ @ yahoo com... com @ @